# Network propagation development

The notebook currently covers how results from a AnnData/MuData object can be added to a cpr_graph to setup network-based inference. The general strategy is to:
1. pull out a pd.DataFrame containing feature-level measures of interest along with feature metadata.
2. These are then mapped on the species ids in an sbml_dfs model by shared on ontology, disambiguated (to handle mapping of multiple features to the same s_id), and s_id-indexed results are embedded in the sbml_dfs as a table in species_data
3. attributes of interrest are then passed from the sbml_dfs model into the graph.

This example uses real MuData results but only a small sbml_dfs object which has uniprot but not ENSG identifiers. This makes things easy to work with but a genome-scale graph will need to be used for a real analysis.

Reflecting on the current functionality,

(1) is not too hard but the interface can probably be cleaned up as we should have a function which applies 1-3 in a single call.
(2) is in pretty good shape following a LOT of new functionality being added to napistu-py for handling many-to-one mappings and wide/nested formats for identifiers.
(3) will need some better functionality since the reaction_attrs syntax is pretty cryptic but the core functionality is all there.

Next, steps will be develop basic PPR functionality.

In [1]:
import os

import pandas as pd

from napistu.ingestion import sbml
from napistu import sbml_dfs_core
from napistu import mechanism_matching

In [2]:
# utils

from typing import Dict
from mudata import MuData

def split_varm_by_modality(mdata: MuData, varm_key: str = 'LFs') -> Dict[str, pd.DataFrame]:
    """
    Split a varm matrix by modality and join with corresponding var tables.
    
    Parameters
    ----------
    mdata : mudata.MuData
        MuData object containing multiple modalities and a varm matrix
    varm_key : str, optional
        Key in mdata.varm to split by modality. Default is 'LFs'
        
    Returns
    -------
    Dict[str, pd.DataFrame]
        Dictionary with modality names as keys and DataFrames as values.
        Each DataFrame contains the var attributes and corresponding matrix values
        for that modality.
        
    Examples
    --------
    >>> # Split latent factors
    >>> modality_lfs = split_varm_by_modality(mdata, varm_key='LFs')
    >>> transcriptomics_lfs = modality_lfs['transcriptomics']
    >>> proteomics_lfs = modality_lfs['proteomics']
    >>>
    >>> # Split other varm matrix
    >>> modality_other = split_varm_by_modality(mdata, varm_key='other_matrix')
    """
    if varm_key not in mdata.varm:
        raise ValueError(f"No '{varm_key}' matrix found in varm")
    
    # Get the matrix and ensure it has the right index
    matrix = pd.DataFrame(
        mdata.varm[varm_key],
        index=mdata.var_names,
        columns=[f'{varm_key}{i+1}' for i in range(mdata.varm[varm_key].shape[1])]
    )
    
    # Initialize results dictionary
    results: Dict[str, pd.DataFrame] = {}
    
    # Process each modality
    for modality in mdata.mod.keys():
        # Get the var_names for this modality
        mod_vars = mdata.mod[modality].var_names
        
        # Extract matrix values for this modality
        mod_matrix = matrix.loc[mod_vars]
        
        # Get var table and ensure index matches
        mod_var = mdata.mod[modality].var.copy()
        
        # Verify index alignment
        if not mod_var.index.equals(mod_matrix.index):
            raise ValueError(
                f"Index mismatch in {modality}: var table and matrix subset have different indices"
            )
        
        # Join with var table on index
        mod_results = pd.concat([mod_var, mod_matrix], axis=1)
        
        # Store in results
        results[modality] = mod_results
    
    return results


DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


In [3]:
PATH_TO_TEST_DATA = os.path.expanduser("~/Desktop/GITHUB/napistu/lib/napistu-py/src/tests/test_data")
example_pathway = os.path.join(PATH_TO_TEST_DATA, "reactome_glucose_metabolism.sbml")
assert os.path.exists(example_pathway)

In [4]:
sbml_dfs = sbml_dfs_core.SBML_dfs(sbml.SBML(example_pathway))

species_identifiers = sbml_dfs.get_identifiers("species").query("bqb == 'BQB_IS'").query("ontology != 'reactome'")

INFO:napistu.utils:creating an edgelist linking index levels s_id, entry and linking it to levels defined by ontology, identifier
DEBUG:napistu.utils:label is not defined in table_schema; adding a constant (1)


In [5]:
# lets load the Forny results so we trying adding a few different types of tables to the sbml_dfs

import mudata as md
OPTIMAL_MODEL_H5MU_PATH = "/tmp/mofa_optimal_model.h5mu"

mdata = md.read_h5mu(OPTIMAL_MODEL_H5MU_PATH)


DEBUG:h5py._conv:Creating converter from 3 to 5
/Users/maya/Desktop/GITHUB/napistu/lib/napistu-scrapyard/applications/forny_2023/.venv/lib/python3.11/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/Users/maya/Desktop/GITHUB/napistu/lib/napistu-scrapyard/applications/forny_2023/.venv/lib/python3.11/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_a

In [6]:
# results from var
var_level_results = mdata["proteomics"].var[["effect_case", "qval_case"]].copy()
var_level_results.index.name = "feature_id"
var_level_results['uniprot'] = var_level_results.index.to_series()

mechanism_matching.bind_wide_results(
    sbml_dfs,
    var_level_results,
    "var_level_results",
    ontologies = {"uniprot"},
    dogmatic = False,
    verbose = True
)

sbml_dfs.species_data["var_level_results"]

INFO:napistu.sbml_dfs_utils:Running in non-dogmatic mode - genes, transcripts, and proteins will be merged if possible.
/Users/maya/Desktop/GITHUB/napistu/lib/napistu-scrapyard/applications/forny_2023/.venv/lib/python3.11/site-packages/napistu/sbml_dfs_core.py:1264: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  promiscuous_component_identifiers = pd.Series(
DEBUG:napistu.mechanism_matching:Validated ontology columns: {'uniprot'}
INFO:napistu.mechanism_matching:Using columns as results: ['feature_id', 'qval_case', 'effect_case']
DEBUG:napistu.mechanism_matching:Final long format shape: (4788, 5)
DEBUG:napistu.mechanism_matching:Matching 4788 features to 98 species for ontology uniprot
INFO:napistu.mechanism_matching:Found 57 total matches across 1 ontologies
INFO:napistu.mechanism_matching:1.3% of feature_ids are present one or more times in the output (57/4513)
INFO:napi

,effect_case,qval_case,feature_id
s_id,,,
S00000006,0.064783,0.996212,1512
S00000012,0.059317,0.996212,4425
S00000013,0.290775,0.996212,2156
S00000015,0.079570,0.996212,1513
S00000016,0.021029,0.996212,"4488,541"
S00000019,0.047774,0.996212,1767
S00000022,0.202818,0.996212,984
S00000031,0.165435,0.996660,"1034,833,906"
S00000033,0.083939,0.996212,730


In [7]:
# merge factors with metadata
import utils
mofa_dfs_dict = utils.split_varm_by_modality(mdata)

modality = "transcriptomics"

mofa_df_list = list()
for modality in mofa_dfs_dict.keys():

    modality_pk = mofa_dfs_dict[modality].index.name
    filter_col = [col for col in mofa_dfs_dict[modality] if col.startswith('LF')]
    modality_df = mofa_dfs_dict[modality][filter_col].copy()
    modality_df.index.name = "feature_id"
    modality_df[modality_pk] = modality_df.index.to_series()
    modality_df["modality"] = modality

    mofa_df_list.append(modality_df)

mofa_df = pd.concat(mofa_df_list, axis=0)

mofa_df.groupby("modality").sample(5)


,LFs1,LFs2,LFs3,LFs4,LFs5,LFs6,LFs7,LFs8,LFs9,LFs10,...,LFs24,LFs25,LFs26,LFs27,LFs28,LFs29,LFs30,ensembl_gene,modality,uniprot
feature_id,,,,,,,,,,,,,,,,,,,,,
Q15056,-0.056613,0.024216,0.043051,0.002981,0.017840,0.038009,0.030697,-0.282958,-0.003660,-0.069065,...,-0.001474,-0.007024,0.001038,0.041995,-0.001967,0.000567,0.001311,NaN,proteomics,Q15056
Q9Y295,-0.065844,0.001523,0.204267,0.008881,0.045325,-0.002938,0.036474,-0.148175,-0.027723,-0.266543,...,-0.009055,-0.002009,0.002134,-0.136886,-0.005834,0.000696,0.003809,NaN,proteomics,Q9Y295
Q13228,0.150933,0.094928,-0.282249,0.001178,0.020215,-0.042123,-0.036351,-0.240952,0.018441,-0.011496,...,0.001096,0.000060,-0.004922,0.060077,-0.003805,0.008359,-0.004654,NaN,proteomics,Q13228
Q9BY32,-0.024243,0.093431,0.006565,-0.001410,0.000312,-0.006907,-0.001519,-0.049320,0.016266,0.000917,...,0.015919,0.010374,-0.006496,0.076231,-0.001936,-0.018307,0.000494,NaN,proteomics,Q9BY32
Q08211,-0.001130,0.054894,0.000127,-0.003526,0.000936,0.015544,0.010073,0.151456,-0.009517,0.004257,...,0.034676,-0.004222,-0.023882,0.143186,0.006172,0.008225,0.000093,NaN,proteomics,Q08211
ENSG00000100567,0.016061,0.039098,0.000143,-0.064764,0.025970,0.066756,-0.060957,0.003253,-0.036549,0.000174,...,-0.113759,0.141184,0.143617,-0.170461,0.073662,-0.003676,0.082745,ENSG00000100567,transcriptomics,NaN
ENSG00000175727,0.005928,-0.089782,0.000036,0.046057,-0.110435,0.007074,0.144012,-0.000737,0.124377,-0.000796,...,-0.061974,-0.133692,0.147478,0.101943,-0.002035,0.150796,-0.362851,ENSG00000175727,transcriptomics,NaN
ENSG00000056972,0.133554,0.136796,0.000035,-0.057148,0.035994,-0.095350,-0.031473,-0.000619,0.121508,-0.000028,...,-0.086435,0.036297,-0.038404,0.248996,-0.155451,-0.027898,-0.005025,ENSG00000056972,transcriptomics,NaN
ENSG00000065491,-0.009292,-0.049694,-0.000006,0.028420,-0.047005,0.057724,0.001863,-0.009377,-0.035145,0.000034,...,0.096928,0.097767,0.018235,0.132387,-0.157928,0.061822,-0.155265,ENSG00000065491,transcriptomics,NaN


In [8]:
mechanism_matching.bind_wide_results(
    sbml_dfs,
    mofa_df,
    "mudata_varm_results",
    ontologies = {"uniprot", "ensembl_gene"},
    dogmatic = False,
    verbose = True
)

sbml_dfs.species_data["mudata_varm_results"]

INFO:napistu.sbml_dfs_utils:Running in non-dogmatic mode - genes, transcripts, and proteins will be merged if possible.


/Users/maya/Desktop/GITHUB/napistu/lib/napistu-scrapyard/applications/forny_2023/.venv/lib/python3.11/site-packages/napistu/sbml_dfs_core.py:1264: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  promiscuous_component_identifiers = pd.Series(
DEBUG:napistu.mechanism_matching:Validated ontology columns: {'ensembl_gene', 'uniprot'}
INFO:napistu.mechanism_matching:Using columns as results: ['LFs14', 'LFs16', 'LFs2', 'LFs17', 'LFs13', 'feature_id', 'LFs5', 'LFs29', 'LFs26', 'LFs3', 'LFs22', 'LFs20', 'LFs24', 'LFs15', 'LFs9', 'LFs10', 'LFs28', 'LFs6', 'LFs19', 'LFs7', 'modality', 'LFs12', 'LFs18', 'LFs11', 'LFs25', 'LFs30', 'LFs1', 'LFs27', 'LFs21', 'LFs23', 'LFs8', 'LFs4']
DEBUG:napistu.mechanism_matching:Final long format shape: (13922, 34)
DEBUG:napistu.mechanism_matching:Matching 4788 features to 98 species for ontology uniprot
INFO:napistu.mechanism_matching:Found 57 total 

,LFs1,LFs2,LFs3,LFs4,LFs5,LFs6,LFs7,LFs8,LFs9,LFs10,...,LFs23,LFs24,LFs25,LFs26,LFs27,LFs28,LFs29,LFs30,modality,feature_id
s_id,,,,,,,,,,,,,,,,,,,,,
S00000006,0.051530,0.036362,-0.041906,-0.001347,-0.013508,0.001565,-0.021276,-0.244355,-0.031964,0.057272,...,-0.191047,0.075975,-0.006429,0.009816,0.074251,0.010391,0.000629,-0.004023,proteomics,10646
S00000012,0.034465,0.115698,-0.047133,-0.001525,-0.006179,0.015768,0.042666,0.118975,0.002245,0.035891,...,0.153040,-0.007040,0.000832,0.005247,0.003451,0.000202,0.001349,-0.002890,proteomics,13559
S00000013,0.111319,-0.002571,-0.073818,-0.000615,-0.066941,0.040771,-0.002893,0.120330,-0.024400,0.337955,...,-0.075491,-0.005003,0.001831,-0.016633,-0.083339,-0.000686,0.003954,-0.002430,proteomics,11290
S00000015,0.010020,0.043131,-0.000815,0.000010,-0.032613,0.026158,0.044396,0.171346,0.001049,0.003718,...,-0.071206,0.054874,-0.052803,-0.000263,-0.014903,0.000827,0.036917,-0.002143,proteomics,10647
S00000016,0.099801,0.064125,-0.015871,-0.001283,-0.085869,0.002985,0.009716,0.216461,-0.005344,0.003299,...,-0.044812,0.032438,-0.001762,0.001995,-0.094416,-0.005864,-0.006293,-0.005239,proteomics,"13622,9675"
S00000019,0.064609,0.006714,-0.110616,-0.017464,-0.020428,-0.032641,-0.003856,0.083932,0.000541,-0.054995,...,-0.121061,-0.032175,-0.006698,-0.002780,0.095618,0.002672,0.074969,-0.004676,proteomics,10901
S00000022,0.034890,0.007629,-0.122095,-0.001708,-0.076321,0.061220,-0.055165,-0.110623,0.014309,0.324787,...,-0.065491,-0.016958,-0.011363,-0.004861,-0.308404,0.006624,-0.031057,0.000834,proteomics,10118
S00000031,0.014932,-0.100028,-0.056794,-0.015458,-0.039523,0.024284,-0.102462,-0.278852,-0.129555,0.008516,...,-0.225068,0.005913,-0.008280,-0.010984,0.235486,0.014511,0.010286,0.005220,proteomics,"10040,10168,9967"
S00000033,0.055625,-0.149947,0.003432,-0.031034,-0.020317,0.054525,-0.158043,-0.283876,-0.124461,0.013506,...,-0.285335,0.016808,-0.004187,-0.003736,0.322549,0.018490,0.048649,-0.000388,proteomics,9864


In [9]:
from napistu.network import net_create

# now we can pass these species_data attributes to the graph

reaction_graph_attrs = {
    "species": {
        "LFs5": {
            "table": "mudata_varm_results",
            "variable": "LFs5",
            "trans": "identity",
        },
        "effect_case": {
            "table": "var_level_results",
            "variable": "effect_case",
            "trans": "identity",
        },
    },
}

cpr_graph = net_create.create_cpr_graph(
    sbml_dfs,
    directed=True,
    graph_type="regulatory"
)

# add species attributes
# TO DO - this is definitely not a utility function
graph_w_annotations = net_create._add_graph_species_attribute(
    cpr_graph,
    sbml_dfs,
    species_graph_attrs = reaction_graph_attrs,
)


INFO:napistu.network.net_create:Organizing all network nodes (compartmentalized species and reactions)
INFO:napistu.network.net_create:Formatting edges as a regulatory graph
INFO:napistu.network.net_create:Formatting 250 reactions species as tiered edges.
INFO:napistu.network.net_create:Adding additional attributes to edges, e.g., # of children and parents.
INFO:napistu.network.net_create:Done preparing regulatory graph
INFO:napistu.network.net_create:Adding reversibility and other meta-data from reactions_data
INFO:napistu.network.net_create:No reactions annotations provided in "graph_attrs"; returning None
INFO:napistu.network.net_create:Creating reverse reactions for reversible reactions on a directed graph
INFO:napistu.network.net_create:Formatting cpr_graph output
INFO:napistu.network.net_create:Adding meta-data from species_data
INFO:napistu.network.net_create:Adding new attribute LFs5 to vertices
INFO:napistu.network.net_create:Adding new attribute effect_case to vertices


In [10]:
from napistu import utils as napistu_utils

napistu_utils.style_df(graph_w_annotations.get_vertex_dataframe().sort_values("LFs5").head(5))

,name,node_name,node_type,LFs5,effect_case
vertex ID,,,,,
26,species_372469,"SLC25A12,13 [mitochondrial inner membrane]",species,-0.086,0.021
96,species_6798333,BPGM dimer [cytosol],species,-0.084,-0.031
36,species_70499,pyruvate carboxylase holoenzyme [mitochondrial matrix],species,-0.076,0.203
18,species_376856,SLC25A11 homodimer [mitochondrial inner membrane],species,-0.067,0.291
58,species_70594,GOT2 dimer [mitochondrial matrix],species,-0.051,0.043
